In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import plotly
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

UFC_FINAL = pd.read_csv("UFC_FINAL.csv")
# tackling imbalance issue
theMin = UFC_FINAL["Winner"].value_counts().min()
minority = UFC_FINAL[UFC_FINAL["Winner"]=="Blue"].iloc[0:theMin]
undersampleMaj = UFC_FINAL[UFC_FINAL["Winner"]=="Red"].iloc[0:theMin]
UFC_FINAL = pd.concat([minority, undersampleMaj], axis=0)
UFC_FINAL = UFC_FINAL.sample(frac=1).reset_index(drop=True)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

# encode blue and red as 1 and 0
UFC_FINAL["Winner"] = UFC_FINAL["Winner"].replace("Blue", 1)
UFC_FINAL["Winner"] = UFC_FINAL["Winner"].replace("Red", 0)

# separate labels
X = UFC_FINAL.drop("Winner", axis=1).values
y = UFC_FINAL["Winner"].values

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=101)

# scale
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape} | X_test shape: {X_test.shape} | y_train shape: {y_train.shape} | y_test shape: {y_test.shape}")

model = Sequential()

model.add(Dense(units=42, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=42, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=42, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=20, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop]
          )

model_loss = pd.DataFrame(model.history.history)

model_loss.plot()
model_loss["val_accuracy"].max()
#

X_train shape: (2239, 42) | X_test shape: (747, 42) | y_train shape: (2239,) | y_test shape: (747,)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2239 samples, validate on 747 samples
Epoch 1/600
2239/2239 [==============================] - 2s 782us/sample - loss: 0.6982 - acc: 0.5154 - val_loss: 0.6930 - val_acc: 0.5033
Epoch 2/600
2239/2239 [==============================] - 0s 157us/sample - loss: 0.6968 - acc: 0.5163 - val_loss: 0.6937 - val_acc: 0.4913
Epoch 3/600
2239/2239 [==============================] - 0s 161us/sample - loss: 0.6961 - acc: 0.4837 - val_loss: 0.6930 - val_acc: 0.5127
Epoch 4/600
2239/2239 [==============================] - 0s 164us/sample - loss: 0.6939 - acc: 0.5087 - val_loss: 0.6929 - val_acc: 0.5100
Epoch 5/600
2239/2239 [==============================] - 0s 156us/sample - loss: 0.69

KeyboardInterrupt: 